In [1]:
!pip install sklearn_crfsuite
!pip install -U 'scikit-learn<0.24'
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 5.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sklearn_crfsuite
import pandas as pd
import numpy as np
import scipy.stats
import sklearn
import joblib
import nltk
import joblib

from sklearn_crfsuite.metrics import flat_classification_report, flat_f1_score
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import scorers

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df_aditamento = pd.read_csv('https://raw.githubusercontent.com/brunoedcf/data_crf_training/main/aditamento_contratual.csv')

In [6]:
class CRF_Aditamento():

  def __init__(self):

    self.x = None
    self.y = None

    self.crf = sklearn_crfsuite.CRF(
      algorithm = 'lbfgs',
      c1=0.203,
      c2=0.032,
      max_iterations=100,
      all_possible_transitions=True
    )

  def get_features(self, sentence):
        
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                # Palavra atual
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                # Uma palavra antes
                'word_before': '' if i == 0 else sentence[i-1].lower(),
                'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
                'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
                'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
                # Uma palavra depois
                'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
                'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
                'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
                'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features

  def load(self, data_frame):

    self.x = []
    self.y = []

    for i, row in enumerate(data_frame['treated_text']):
      self.x.append(word_tokenize(data_frame['treated_text'][i]))
      self.y.append(data_frame['IOB'][i].split())

    for i in range(len(self.x)):
        self.x[i] = self.get_features(self.x[i])


  def train(self):

    self.crf.fit(self.x, self.y)

    joblib.dump(self.crf, "aditamento.pkl")


In [8]:
model = CRF_Aditamento()
model.load(df_aditamento)

In [9]:
model.train()